In [39]:
from cobra import Reaction, Metabolite
from cobra.io import read_sbml_model
from cobra.io import write_sbml_model

The metabolism of V. natriegens ATCC 14048 was investigated by Long et al. by carbon-13 flux analysis in order to determine intracellular metabolic fluxes of this organism (Long et al. 2017). These analyses were performed in a minimal media (M9) with glucose as an carbon source. For this reason, the refseq of V. natriegens ATCC 14048 from NCBI is chosen for the construction of the GSM in order to have as much simularity as possible to the isolate used. (TAKEN FROM GROUP7-2020)
We now switch to LB media to compare results (SAY PAPER WHERE PROPOSES LB AS BEST MEDIA). 

In [70]:
modelLB=read_sbml_model('Models/Vibrio-natriegens-ATCC_14048-LB.xml')

In [71]:
print("Number of reactions:", len(modelLB.reactions))
print("Number of metabolites:", len(modelLB.metabolites))
print("Number of genes included in the model:", len(modelLB.genes))

Number of reactions: 2720
Number of metabolites: 1730
Number of genes included in the model: 1480


Percentage of coding genes covered (from: https://www.ncbi.nlm.nih.gov/nuccore/520916752?report=genbank)

In [72]:
print("Percentage of genes included in the model compared to the number of existing coding genes in Vibrio natrigens(4513 genes):",round(((len(modelLB.genes)*100)/4513), 3),"%")

Percentage of genes included in the model compared to the number of existing coding genes in Vibrio natrigens(4513 genes): 32.794 %


We start this study checking the initial growth rate, for a later comparison with the optimised model we will create.

In [73]:
print("Maximal growth rate of Vibrio natriegens in LB medium:", round(modelLB.optimize().objective_value, 3))

Maximal growth rate of Vibrio natriegens in LB medium: 2.813


First, one has to look for the start point of the violacein pathway, which is known to be tryptophan synthase.

In [74]:
modelLB.reactions.TRPS2 # Reaction before L-trp -> IPA Imine

Reaction identifier,TRPS2
Name,Tryptophan synthase (indole)
Memory address,0x07fbd41324dc0
Stoichiometry,indole_c + ser__L_c --> h2o_c + trp__L_c Indole + L-Serine --> H2O H2O + L-Tryptophan
GPR,WP_217999032_1 or ( WP_014232411_1 and WP_020335573_1 )
Lower bound,0.0
Upper bound,1000.0


After finding the starting point of the violacein pathway, one has to make sure that side reactions of violacein production are leveled in the model. This reactions involve chlorines, hemo groups and pyruvate. 
The proper study can be found in *Side_reactions-LB.ipynb*, where it is ensured that the model is properly balanced, and the species would not die.

Once everything is properly balanced, one can start adding violacein production pathway.

First, one must define all metabolites used that are not native to *Vibrio Natrigens*.

In [75]:
# --- For violacein production ---
ipa_c=Metabolite(id='ipa_c', formula='C11H10N2O2', name='2-Imino-3-(indol-3-yl)propanoate', compartment='c')
ipa_dimer_c=Metabolite(id='ipa_dimer_c', compartment='c')
ptdv_c=Metabolite(id='ptdv_c', formula='C21H15N3O2', name='Protodeoxyviolaceinate', compartment='c')
ptv_c=Metabolite(id='ptv_c', formula='C21H15N3O3', name='Protoviolaceinate', compartment='c')
va_c=Metabolite(id='va_c', formula='C21H15N3O4', name='Violaceinate', compartment='c')
v_c=Metabolite(id='v_c', formula='C20H13N3O3', name='Violacein', compartment='c')
# --- Side reactions ---
cp_c=Metabolite(id='cp_c', formula='C22H15N3O4', name='Chromopyrrolate', compartment='c')
dva_c=Metabolite(id='dva_c', formula='C21H15N3O3', name='Deoxyviolaceinate', compartment='c')
dv_c=Metabolite(id='dv_c', formula='C20H13N3O2', name='Deoxyviolacein', compartment='c')
pdv_c=Metabolite(id='pdv_c',formula='C20H13N3O', name='Prodeoxyviolacein', compartment='c')
pv_c=Metabolite(id='pv_c',formula='C20H13N3O2', name='Proviolacein', compartment='c')

Later, the reactions from tryptophan to violacein have to be built.

In [76]:
Reaction_VioA_FAD=Reaction('IPA_imine_VioA_FAD') # L-tryptophan + FAD <=> IPA imine + FADH2

Reaction_VioA_FAD.add_metabolites({modelLB.metabolites.trp__L_c: -1, # L-tryptophan
                                 modelLB.metabolites.fad_c: -1, # FAD
                                 ipa_c: 1, # IPA imine
                                 modelLB.metabolites.fadh2_c: 1 # FADH2
                                })

In [77]:
Reaction_VioA_O2=Reaction('IPA_imine_VioA_O2') # L-Tryptophan + Oxygen <=> IPA imine + Hydrogen peroxide

Reaction_VioA_O2.add_metabolites({modelLB.metabolites.trp__L_c: -1,   # L-tryptophan
                                 modelLB.metabolites.o2_c: -1,            # Oxygen
                                 ipa_c: 1,                               # IPA imine
                                 modelLB.metabolites.h2o2_c: 1            # Hydrogen peroxide
                                })

In [78]:
Reaction_VioB=Reaction('IPA_imine_dimer_VioB') #  # 2 IPA imine + Fe2+ => IPA imine dimer + Fe3+ 

Reaction_VioB.add_metabolites({ipa_c: -2,               # IPA imine
                                 modelLB.metabolites.fe2_c: -1,    # Fe2+
                                 ipa_dimer_c: 1,                  # IPA imine dimer
                                 modelLB.metabolites.fe3_c: 1      # Fe3+ 
                                })

In [79]:
Reaction_Chromopyrrolate=Reaction('Chromopyrrolate') # IPA imine dimer <=> Chromopyrollic acid + NH3 

Reaction_Chromopyrrolate.add_metabolites({ipa_dimer_c: -1,         # IPA imine
                                 cp_c: 1,                   # Chromopyrollic acid
                                 modelLB.metabolites.nh3_c: 1       # NH3 
                                })

In [80]:
Reaction_VioE=Reaction('Protodeoxyviolaceinate_VioE') # IPA imine dimer <=> Protodeoxyviolaceinic acid + CO2 + NH3

Reaction_VioE.add_metabolites({ipa_dimer_c: -1,      # IPA imine dimer
                                 modelLB.metabolites.co2_c: 1,          # CO2
                                 ptdv_c: 1,                            # Protodeoxyviolaceinic acid
                                 modelLB.metabolites.nh3_c: 1           # NH3 
                                })

In [81]:
Reaction_Prodeoxyviolacein=Reaction('Prodeoxyviolacein') # Protodeoxyviolaceinic acid + O2 <=> Prodeoxyviolacein + CO2 +H2O

Reaction_Prodeoxyviolacein.add_metabolites({ptdv_c: -1,                # Protodeoxyviolaceinic acid
                                 modelLB.metabolites.o2_c: -1,          # Oxygen         
                                 pdv_c: 1,                             # Prodeoxyviolacein
                                 modelLB.metabolites.co2_c: 1,          # CO2
                                 modelLB.metabolites.h2o_c: 1           # H2O
                                })

In [82]:
Reaction_Deoxyviolaceinate_VioC=Reaction('Deoxyviolaceinate_VioC') # Protodeoxyviolaceinic acid + Oxygen + NADPH + H+ <=> Deoxyviolaceinic acid + NADP+ + H2O

Reaction_Deoxyviolaceinate_VioC.add_metabolites({ptdv_c: -1,           # Protodeoxyviolaceinic acid
                                 modelLB.metabolites.nadph_c: -1,       # NADPH
                                 modelLB.metabolites.o2_c: -1,          # Oxygen
                                 modelLB.metabolites.h_c: -1,           # H+           
                                 dva_c: 1,                             # Deoxyviolaceinic acid
                                 modelLB.metabolites.nadp_c: 1,         # NADP+
                                 modelLB.metabolites.h2o_c: 1           # H2O
                                })

In [83]:
Reaction_Deoxyviolacein=Reaction('Deoxyviolacein') # Deoxyviolaceinic acid + Oxygen <=> Deoxyviolacein + CO2 + H2O

Reaction_Deoxyviolacein.add_metabolites({dva_c: -1,                    # Deoxyviolaceinic acid
                                 modelLB.metabolites.o2_c: -1,          # Oxygen         
                                 dv_c: 1,                              # Deoxyviolacein
                                 modelLB.metabolites.co2_c: 1,          # CO2
                                 modelLB.metabolites.h2o_c: 1           # H2O
                                })

In [84]:
Reaction_VioD=Reaction('Protoviolaceinate_VioD') # Protodeoxyviolaceinic acid + NADPH + Oxygen + H+ <=> Protoviolaceinic acid + NADP+ + H2O

Reaction_VioD.add_metabolites({ptdv_c: -1,                             # Protodeoxyviolaceinic acid
                                 modelLB.metabolites.nadph_c: -1,       # NADPH
                                 modelLB.metabolites.o2_c: -1,          # Oxygen
                                 modelLB.metabolites.h_c: -1,           # H+           
                                 ptv_c: 1,                             # Protoviolaceinic acid
                                 modelLB.metabolites.nadp_c: 1,         # NADP+
                                 modelLB.metabolites.h2o_c: 1           # H2O
                                })

In [85]:
Reaction_Proviolacein=Reaction('Proviolacein') # Protoviolaceinic acid + O2 = Proviolacein +CO2 + H2O

Reaction_Proviolacein.add_metabolites({ptv_c: -1,                    # Protoviolaceinic acid
                                 modelLB.metabolites.o2_c: -1,          # Oxygen         
                                 pv_c: 1,                              # Proviolacein
                                 modelLB.metabolites.co2_c: 1,          # CO2
                                 modelLB.metabolites.h2o_c: 1           # H2O
                                })

In [86]:
Reaction_Violaceinate_VioC=Reaction('Violaceinate') # Protoviolaceinic acid + Oxygen + NADPH + H+ <=> Violaceinic acid + NADP+ + H2O

Reaction_Violaceinate_VioC.add_metabolites({ptv_c: -1,                 # Protoviolaceinic acid
                                 modelLB.metabolites.nadph_c: -1,       # NADPH
                                 modelLB.metabolites.o2_c: -1,          # Oxygen
                                 modelLB.metabolites.h_c: -1,           # H+           
                                 va_c: 1,                              # Violaceinic acid
                                 modelLB.metabolites.nadp_c: 1,         # NADP+
                                 modelLB.metabolites.h2o_c: 1           # H2O
                                })

In [87]:
Reaction_VioD=Reaction('Protoviolaceinate_VioD') # Protodeoxyviolaceinic acid + NADPH + Oxygen + H+ <=> Protoviolaceinic acid + NADP+ + H2O

Reaction_VioD.add_metabolites({ptdv_c: -1,                             # Protodeoxyviolaceinic acid
                                 modelLB.metabolites.nadph_c: -1,       # NADPH
                                 modelLB.metabolites.o2_c: -1,          # Oxygen
                                 modelLB.metabolites.h_c: -1,           # H+           
                                 ptv_c: 1,                             # Protoviolaceinic acid
                                 modelLB.metabolites.nadp_c: 1,         # NADP+
                                 modelLB.metabolites.h2o_c: 1           # H2O
                                })

In [88]:
Reaction_Proviolacein=Reaction('Proviolacein') # Protoviolaceinic acid + O2 = Proviolacein +CO2 + H2O

Reaction_Proviolacein.add_metabolites({ptv_c: -1,                    # Protoviolaceinic acid
                                 modelLB.metabolites.o2_c: -1,          # Oxygen         
                                 pv_c: 1,                              # Proviolacein
                                 modelLB.metabolites.co2_c: 1,          # CO2
                                 modelLB.metabolites.h2o_c: 1           # H2O
                                })

In [89]:
Reaction_Violaceinate_VioC=Reaction('Violaceinate') # Protoviolaceinic acid + Oxygen + NADPH + H+ <=> Violaceinic acid + NADP+ + H2O

Reaction_Violaceinate_VioC.add_metabolites({ptv_c: -1,                 # Protoviolaceinic acid
                                 modelLB.metabolites.nadph_c: -1,       # NADPH
                                 modelLB.metabolites.o2_c: -1,          # Oxygen
                                 modelLB.metabolites.h_c: -1,           # H+           
                                 va_c: 1,                              # Violaceinic acid
                                 modelLB.metabolites.nadp_c: 1,         # NADP+
                                 modelLB.metabolites.h2o_c: 1           # H2O
                                })

In [90]:
Reaction_Violacein=Reaction('Violacein') # Violaceinic acid + Oxygen <=> Violacein + CO2 + H2O

Reaction_Violacein.add_metabolites({va_c: -1,                          # Violaceinic acid
                                 modelLB.metabolites.o2_c: -1,          # Oxygen         
                                 v_c: 1,                               # Violacein
                                 modelLB.metabolites.co2_c: 1,          # CO2
                                 modelLB.metabolites.h2o_c: 1           # H2O
                                })

All reactions must be added to the model.

In [91]:
# modelLB.add_reactions([Reaction_VioA_O2]) or modelLB.add_reactions([Reaction_VioA_FAD])
modelLB.add_reactions([Reaction_VioA_FAD])
# --------------------------------------- #
modelLB.add_reactions([Reaction_VioB])
modelLB.add_reactions([Reaction_Chromopyrrolate])
modelLB.add_reactions([Reaction_VioE])
modelLB.add_reactions([Reaction_Prodeoxyviolacein])
modelLB.add_reactions([Reaction_Deoxyviolaceinate_VioC])
modelLB.add_reactions([Reaction_Deoxyviolacein])
modelLB.add_reactions([Reaction_VioD])
modelLB.add_reactions([Reaction_Proviolacein])
modelLB.add_reactions([Reaction_Violaceinate_VioC])
modelLB.add_reactions([Reaction_Violacein])

One must ensure that boundaries are set correctly, and add them if they are not.

In [92]:
modelLB.demands

[]

In [93]:
modelLB.add_boundary(modelLB.metabolites.get_by_id("v_c"), type="demand")

Reaction identifier,DM_v_c
Name,Violacein demand
Memory address,0x07fbd415c14c0
Stoichiometry,v_c --> Violacein -->
GPR,
Lower bound,0
Upper bound,1000.0


In [94]:
modelLB.add_boundary(modelLB.metabolites.get_by_id("dv_c"), type="demand")

Reaction identifier,DM_dv_c
Name,Deoxyviolacein demand
Memory address,0x07fbd415ab280
Stoichiometry,dv_c --> Deoxyviolacein -->
GPR,
Lower bound,0
Upper bound,1000.0


The following step one must do is changing the objective of the model.

In [95]:
modelLB.objective = modelLB.reactions.Violacein

Finally, one must retrieve the maximum production.

In [96]:
violacein_production = modelLB.optimize().objective_value
print('Max. violacein production in LB media [mmol gDW^-1 h^-1]:',violacein_production)

Max. violacein production in LB media [mmol gDW^-1 h^-1]: 3.3333333333333326


The maximum production rate of violacein in *V. natriegens* (LB media) with the violacein pathway from *V. campbellii*, using FAD, is 3.33 mmol gDW-1 h-1.

The new model including violacein pathway with FAD is stored in Models folder. 

In [97]:
# Idented to avoid writing over the file every time. At the end of the project we should unident it :)
### write_sbml_model(modelLB,"Models/model_VN_LB_vio-fad.xml")